# Sumário

[Funções de Ativação](#Funções-de-Ativação)

[Implementação](#Implementação)

[Teste](#Teste)

[Referências](#Referências)

# Imports and Configurações

In [1]:
import numpy as np

# Funções de Ativação

In [35]:
def linear(x, derivative=False):
    return np.ones_like(x) if derivative else x

def sigmoid(x, derivative=False):
    if derivative:
        y = sigmoid(x)
        return y*(1-y)
    return 1.0/(1.0 + np.exp(-x))

def tanh(x, derivative=False):
    if derivative:
        y = tanh(x)
        return 1 - y**2
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

def relu(x, derivative=False):
    if derivative:
        return np.where(x <= 0, 0, 1)
    return np.maximum(0, x)

def leaky_relu(x, derivative=False):
    if derivative:
        return np.where(x <= 0, 0.1, 1)
    return np.where(x < 0, 0.1*x, x)

def gaussian(x, derivative=False):
    if derivative:
        return -2*x*np.exp(-x**2)
    return np.exp(-x**2)

# Funções de custo

In [42]:
def mse(y, y_pred, derivative=False):
    if derivative:
        return np.mean(-(y - y_pred))
    return np.mean((y - y_pred)**2)

def sigmoid_cross_entropy(y, y_pred, derivative=False):
    if derivative:
        pass
    return -np.mean(y*np.log(y_pred) + (1-y)*np.log(1-y_pred))

def softmax_cross_entropy(y, y_pred, derivative=False):
    if derivative:
        return -( y*(1/y_pred) + (1-y)*(1/(1-y_pred)) )
    return -np.mean((y*np.log(y_pred) + (1-y)*np.log(1-y_pred)).sum(axis=1))

# Implementação 

In [43]:
class NeuralNetwork():
    def __init__(self, layers_size, activations, cost_func, learning_rate=1e-3):
        self.layers_size = layers_size
        self.activations = activations
        self.cost_func = cost_func
        self.learning_rate = learning_rate
        self._layers_inp = [] # entrada da função de ativação
        self._layers_out = []
        self.weights = self.__init_weights_and_bias()
            
    def fit(self, x, y, epochs=100, verbose=10):
        for epoch in range(epochs):
            y_pred = self.__feedforward(x)
            self.__backprop(y, y_pred)
            
            if epoch % verbose == 0:
                cost = self.cost_func(y, y_pred)
                print("epoch: {0:=4}/{1} cost: {2}".format(epoch, epochs, cost))
    
    def predict(self, x):
        return self.__feedforward(x)
    
    def __init_weights_and_bias(self):
        weights = []
        for inp_layer, out_layer in zip(self.layers_size[:-1], self.layers_size[1:]):
            weights.append(np.random.randn(out_layer, inp_layer))
        return weights
        
    def __feedforward(self, x):
        self._layers_out.append(x)
        for w, activation in zip(self.weights, self.activations):
            y = np.dot(self._layers_out[-1], w.T)
            self._layers_inp.append(y)
            self._layers_out.append(activation(y))
        return self._layers_out[-1]
    
    def __backprop(self, y, y_pred):
        self._layers_out.pop()
        
        dout = self.cost_func(y, y_pred, derivative=True)
        last_delta = dout
        dweights = []
        for inp, out, w, activation in zip(reversed(self._layers_inp), reversed(self._layers_out), reversed(self.weights), reversed(self.activations)):
            dactivation = activation(inp, derivative=True)*last_delta
            last_delta = np.dot(dactivation, w)
            dweights.append(np.dot(dactivation.T, out))
            
        for i, dw in zip(reversed(range(len(self.weights))), dweights):
            self.weights[i] = self.weights[i] - self.learning_rate*dw

# Teste

In [44]:
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0]).reshape(-1, 1)

print(x.shape, y.shape)

(4, 2) (4, 1)


In [49]:
nn = NeuralNetwork(layers_size=[2, 3, 4, 3, 1], activations=[relu, relu, relu, linear], cost_func=mse, learning_rate=1e-2)

print([w.T.shape for w in nn.weights])
nn.fit(x, y)

[(2, 3), (3, 4), (4, 3), (3, 1)]
epoch:    0/100 cost: 0.46492462674886725
epoch:   10/100 cost: 0.44967891522200815
epoch:   20/100 cost: 0.4321091782287205
epoch:   30/100 cost: 0.4119916082915246
epoch:   40/100 cost: 0.3902213475188104
epoch:   50/100 cost: 0.3697626865216482
epoch:   60/100 cost: 0.3562289335772501
epoch:   70/100 cost: 0.35494434614031584
epoch:   80/100 cost: 0.36801867168497454
epoch:   90/100 cost: 0.39135899778297


# Referências